<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0
1.8.0


Read data created in the previous chapter.

In [4]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon', 'pickuplat', 'dropofflon', 'dropofflat', 'passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header=None, names=CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header=None, names=CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header=None, names=CSV_COLUMNS)



<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [5]:
# Train and eval input functions to read from pandas Dataframe
def make_train_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = df,
        y = df[LABEL],
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000
    )

In [6]:
def make_eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x = df,
        y = df[LABEL],
        batch_size = 128,
        shuffle = False,
        queue_capacity = 1000
    )

Our input function for predictions is the same except we don't provide a label

In [7]:
# Our input function for predictions is the same except we don't provide a label
def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x = df,
        y = None,
        batch_size = 128,
        shuffle = False,
        queue_capacity = 1000)

### Create feature columns for estimator

In [11]:
# Create feature columns for estimator
def make_feature_cols():
    input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
    return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [13]:
# Linear Reg with tf.estimator
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
        feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = tf.estimator.inputs.pandas_input_fn(
        x = df_train,
        y = df_train[LABEL],
        batch_size = 128,
        shuffle = False,
        queue_capacity = 1000
    ))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe76eb857b8>, '_service': None, '_global_id_in_cluster': 0, '_model_dir': 'taxi_trained', '_master': '', '_evaluation_master': '', '_session_config': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_is_chief': True, '_task_id': 0, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_task_type': 'worker'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 24937.602, st

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [14]:
# Evaluate on the validation data
# Defer test data after final model

def print_rmse(model, df):
    metrics = model.evaluate(input_fn = make_eval_input_fn(df))
    print('RMSE on datastet = {}'.format(np.sqrt(metrics['average_loss'])))
    
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-03-07:26:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-61
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-03-07:26:43
INFO:tensorflow:Saving dict for global step 61: average_loss = 109.56537, global_step = 61, loss = 13030.453
RMSE on datastet = 10.467348098754883


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like. Let's use this model for prediction.

In [15]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
    print(items)

# This explains why the RMSE was so high.
# Shows that predicts the same amount for every trip.

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-61
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.771139], dtype=float32)}
{'predictions': array([10.768656], dtype=float32)}
{'predictions': array([10.769515], dtype=float32)}
{'predictions': array([10.76708], dtype=float32)}
{'predictions': array([10.770896], dtype=float32)}
{'predictions': array([10.770668], dtype=float32)}
{'predictions': array([10.769281], dtype=float32)}
{'predictions': array([10.76932], dtype=float32)}
{'predictions': array([10.771063], dtype=float32)}
{'predictions': array([10.768892], dtype=float32)}
{'predictions': array([10.771136], dtype=float32)}
{'predictions': array([10.771305], dtype=float32)}
{'predictions': array([10.764715], dtype=float32)}
{'predictions': array([10.768551], dtype=float32)}
{'predictions': array([10.

{'predictions': array([10.769312], dtype=float32)}
{'predictions': array([10.819965], dtype=float32)}
{'predictions': array([10.95743], dtype=float32)}
{'predictions': array([10.960542], dtype=float32)}
{'predictions': array([10.769654], dtype=float32)}
{'predictions': array([10.770055], dtype=float32)}
{'predictions': array([10.766741], dtype=float32)}
{'predictions': array([10.772491], dtype=float32)}
{'predictions': array([10.765384], dtype=float32)}
{'predictions': array([10.769285], dtype=float32)}
{'predictions': array([10.771458], dtype=float32)}
{'predictions': array([10.768848], dtype=float32)}
{'predictions': array([10.772686], dtype=float32)}
{'predictions': array([10.769486], dtype=float32)}
{'predictions': array([10.769848], dtype=float32)}
{'predictions': array([10.819584], dtype=float32)}
{'predictions': array([10.76881], dtype=float32)}
{'predictions': array([10.865402], dtype=float32)}
{'predictions': array([10.910191], dtype=float32)}
{'predictions': array([10.817111]

{'predictions': array([10.818305], dtype=float32)}
{'predictions': array([10.816935], dtype=float32)}
{'predictions': array([10.770185], dtype=float32)}
{'predictions': array([10.76625], dtype=float32)}
{'predictions': array([10.815875], dtype=float32)}
{'predictions': array([10.768525], dtype=float32)}
{'predictions': array([10.76823], dtype=float32)}
{'predictions': array([10.861286], dtype=float32)}
{'predictions': array([10.818442], dtype=float32)}
{'predictions': array([10.768813], dtype=float32)}
{'predictions': array([10.769192], dtype=float32)}
{'predictions': array([10.817497], dtype=float32)}
{'predictions': array([10.817285], dtype=float32)}
{'predictions': array([10.769288], dtype=float32)}
{'predictions': array([10.962159], dtype=float32)}
{'predictions': array([10.772014], dtype=float32)}
{'predictions': array([10.769644], dtype=float32)}
{'predictions': array([10.770199], dtype=float32)}
{'predictions': array([10.770212], dtype=float32)}
{'predictions': array([10.769282]

{'predictions': array([10.773072], dtype=float32)}
{'predictions': array([10.764465], dtype=float32)}
{'predictions': array([10.768852], dtype=float32)}
{'predictions': array([10.764891], dtype=float32)}
{'predictions': array([10.8129], dtype=float32)}
{'predictions': array([10.771161], dtype=float32)}
{'predictions': array([10.768092], dtype=float32)}
{'predictions': array([10.7691555], dtype=float32)}
{'predictions': array([10.818882], dtype=float32)}
{'predictions': array([10.805746], dtype=float32)}
{'predictions': array([10.811178], dtype=float32)}
{'predictions': array([10.769794], dtype=float32)}
{'predictions': array([10.818241], dtype=float32)}
{'predictions': array([10.770143], dtype=float32)}
{'predictions': array([10.769662], dtype=float32)}
{'predictions': array([10.816685], dtype=float32)}
{'predictions': array([10.77119], dtype=float32)}
{'predictions': array([10.768897], dtype=float32)}
{'predictions': array([10.769684], dtype=float32)}
{'predictions': array([10.768747]

{'predictions': array([10.7696], dtype=float32)}
{'predictions': array([10.769284], dtype=float32)}
{'predictions': array([10.797245], dtype=float32)}
{'predictions': array([10.815685], dtype=float32)}
{'predictions': array([10.815878], dtype=float32)}
{'predictions': array([10.759787], dtype=float32)}
{'predictions': array([10.773403], dtype=float32)}
{'predictions': array([10.961861], dtype=float32)}
{'predictions': array([10.816753], dtype=float32)}
{'predictions': array([10.763697], dtype=float32)}
{'predictions': array([10.964554], dtype=float32)}
{'predictions': array([10.820663], dtype=float32)}
{'predictions': array([10.769069], dtype=float32)}
{'predictions': array([10.768618], dtype=float32)}
{'predictions': array([10.96179], dtype=float32)}
{'predictions': array([10.771033], dtype=float32)}
{'predictions': array([10.81693], dtype=float32)}
{'predictions': array([10.769589], dtype=float32)}
{'predictions': array([10.768173], dtype=float32)}
{'predictions': array([10.770123], 

{'predictions': array([10.763332], dtype=float32)}
{'predictions': array([10.769659], dtype=float32)}
{'predictions': array([10.8165455], dtype=float32)}
{'predictions': array([10.769932], dtype=float32)}
{'predictions': array([10.958302], dtype=float32)}
{'predictions': array([10.770613], dtype=float32)}
{'predictions': array([10.864159], dtype=float32)}
{'predictions': array([10.962161], dtype=float32)}
{'predictions': array([10.963996], dtype=float32)}
{'predictions': array([10.755705], dtype=float32)}
{'predictions': array([10.767222], dtype=float32)}
{'predictions': array([10.769879], dtype=float32)}
{'predictions': array([10.770994], dtype=float32)}
{'predictions': array([10.961149], dtype=float32)}
{'predictions': array([10.770114], dtype=float32)}
{'predictions': array([10.766246], dtype=float32)}
{'predictions': array([10.770183], dtype=float32)}
{'predictions': array([10.770301], dtype=float32)}
{'predictions': array([10.914699], dtype=float32)}
{'predictions': array([10.7700

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [16]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fesh each time

model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
                                  feature_columns = make_feature_cols(),
                                  model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100))

print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe74df0a438>, '_service': None, '_global_id_in_cluster': 0, '_model_dir': 'taxi_trained', '_master': '', '_evaluation_master': '', '_session_config': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_is_chief': True, '_task_id': 0, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_num_worker_replicas': 1, '_tf_random_seed': None, '_task_type': 'worker'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 219605.03, st

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [17]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [18]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-03-07:37:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-03-07:37:42
INFO:tensorflow:Saving dict for global step 6071: average_loss = 112.78394, global_step = 6071, loss = 14330.118
RMSE on datastet = 10.619978904724121


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License